**Text Summarization using the TextRank Algorithm**

In [1]:
#importing required python libraries
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') 
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
#importing excel files using pandas
df = pd.read_excel('Test.xlsx',)

In [3]:
 #few ines in our data
df.head()  

,Inforrmation
0,Acnesol Gel is an antibiotic that fights bacte...
1,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...


In [4]:
df['Inforrmation'][0]

'Acnesol Gel is an antibiotic that fights bacteria. It is used to treat acne, which appears as spots or pimples on your face, chest or back. This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor. You should normally wash and dry the affected area before applying a thin layer of the medicine. It should not be applied to broken or damaged skin. Avoid any contact with your eyes, nose, or mouth. Rinse it off with water if you accidentally get it in these areas. It may take several weeks for your symptoms to improve, but you should keep using this medicine regularly. Do not stop using it as soon as your acne starts to get better. Ask your doctor when you should stop treatment.Common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people. These are usually temporary and resolve on their own. Consult your doctor if they bother you or do no

In [5]:
df['Inforrmation'][2]

'Augmentin 625 Duo Tablet is a penicillin-type of antibiotic that helps your body fight infections caused by bacteria. It is used to treat infections of the lungs (e.g., pneumonia), ear, nasal sinus, urinary tract, skin and soft tissue. It will not work for viral infections such as the common cold.Augmentin 625 Duo Tablet is best taken with a meal to reduce the chance of a stomach upset. You should take it regularly at evenly spaced intervals as per the schedule prescribed by your doctor. Taking it at the same time every day will help you to remember to take it. The dose will depend on what you are being treated for, but you should always complete a full course of this antibiotic as prescribed by your doctor. Do not stop taking it until you have finished, even when you feel better. If you stop taking it early, some bacteria may survive and the infection may come back or worsen.The most common side effects of this medicine include vomiting, nausea, and diarrhea. These are usually mild b

In [6]:
# I break the text into individual sentences and used the sent_tokenize( ) function of the nltk library to do this.
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['Inforrmation']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] 

In [7]:
#few elements of the list sentences
sentences[:5]

['Acnesol Gel is an antibiotic that fights bacteria.',
 'It is used to treat acne, which appears as spots or pimples on your face, chest or back.',
 'This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor.',
 'You should normally wash and dry the affected area before applying a thin layer of the medicine.',
 'It should not be applied to broken or damaged skin.']

In [8]:
'''GloVe word embeddings are vector representation of words. These word embeddings will be used to create vectors for our sentences. 
We could have also used the Bag-of-Words or TF-IDF approaches to create features for our sentencesGloVe word embeddings are vector representation 
of words. These word embeddings will be used to create vectors for our sentences. We could have also used the Bag-of-Words or TF-IDF approaches 
to create features for our sentences'''

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-06-27 13:25:39--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-27 13:25:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-27 13:25:39--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [9]:
# Extraction word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [10]:
len(word_embeddings)
# i.e. We now have word vectors for 400,000 different terms stored in the dictionary – ‘word_embeddings’.

400000

In [11]:
# It is always a good practice to make your textual data noise-free as much as possible.
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

In [12]:
# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
# Get rid of the stopwords (commonly used words of a language – is, am, the, of, in, etc.) present in the sentences.
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [16]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [17]:
# Vector Representation of Sentences
# Extract word vectors

word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [18]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
len(sentence_vectors)

In [20]:
'''The next step is to find similarities between the sentences, and we will use the cosine similarity approach for this challenge. 
Let’s create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.'''
sim_mat = np.zeros([len(sentences), len(sentences)])

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [23]:
# Applying PageRank Algorithm
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [24]:
# Finally, it’s time to extract the top N sentences based on their rankings for summary generation

ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [36]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

Also, inform the doctor if you have ever had bloody diarrhea caused by taking antibiotics or if you are using any other medicines to treat skin conditions.
Take it regularly and do not stop taking the medicine even if you get better until that doctor tell you it is alright to stop.The most common side effect of this medicine is stomach pain.
It is advised not to use it for more than 14 days without doctor consultation.The most common side effects of this medicine include vomiting, nausea, and stomach upset.
Your doctor may help with ways to reduce or prevent these symptoms by reducing the dose or prescribing an alternative medicine.To make sure the medicine is safe for you, before taking this medicine, let your doctor know all the other medicines you are taking.
Avoid drinking alcohol while taking this medicine as it can make sleepiness worse.Never support self-medication or recommend your medicine to another person.
Consult your doctor if you find these side effects worry you or persi